<a href="https://colab.research.google.com/github/Darshan-Bitla7/Detecting-Depression-through-Tweets/blob/main/Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/SentimentAnalysisDataset2.csv', on_bad_lines='skip') 

In [ ]:
df=df.drop(['ItemID','SentimentSource'], axis=1)

In [ ]:
df['SentimentText'] = df['SentimentText'].apply(lambda x: x.lower())
df['SentimentText'] = df['SentimentText'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [ ]:
df


,Sentiment,SentimentText
0,0,is so sad for my apl friend
1,0,i missed the new moon trailer
2,1,omg its already 730 o
3,0,omgaga im sooo im gunna cry ive be...
4,0,i think mi bf is cheating on me ...
...,...,...
1578607,1,zzzzzz finally night tweeters
1578608,1,zzzzzzz sleep well people
1578609,0,zzzzzzzzzz wait no i have homework
1578610,0,zzzzzzzzzzzzz meh what am i doing up again


In [ ]:
print(df[ df['Sentiment'] == 1].size)
print(df[ df['Sentiment'] == 0].size)
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['SentimentText'].values)
X = tokenizer.texts_to_sequences(df['SentimentText'].values)
X = pad_sequences(X)

1580354
1576870


In [ ]:
!pip install keras
from keras.layers import Dropout

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
embed_dim = 50
lstm_out = 80
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 50)            100000    
                                                                 
 lstm (LSTM)                 (None, 80)                41920     
                                                                 
 dense (Dense)               (None, 2)                 162       
                                                                 
Total params: 142,082
Trainable params: 142,082
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y = pd.get_dummies(df['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1057670, 40) (1057670, 2)
(520942, 40) (520942, 2)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

33053/33053 - 990s - loss: 0.4494 - accuracy: 0.7881 - 990s/epoch - 30ms/step


In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

16233/16233 - 151s - loss: 0.4288 - accuracy: 0.7999 - 151s/epoch - 9ms/step
score: 0.43
acc: 0.80


In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    #print(data['text'].iloc[-validation_size+x])
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    #print(result)
    #print(Y_validate[x])
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    #print(np.argmax(Y_validate[x]))
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
    #print(np.argmax(result) == np.argmax(Y_validate[x]))
    #print()
    #print()
    
#print("pos_cnt", pos_cnt)
#print("neg_cnt", neg_cnt)
#print("pos_correct", pos_correct)
#print("neg_correct", neg_correct)

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 1s - 520ms/epoch - 520ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 15ms/epoch - 15ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 1